In [3]:
import pandas as pd
import numpy as np
import glob

In [4]:
#Function to create a column that combines QAN with the word QAN to help with merging
def QAN_merge_col (df,col):
    return "QAN " + str(df[col])

In [5]:
#Function to create a column that combines QAN with the word QAN to help with merging
def QAN_grade_col (df):
    return str(df['QAN Merge']) + " " + str(df['Pass Grades'])

In [6]:
#Function to create a column that combines QAN with the student grade to enable merging
def QAN_student_grade_col (df):
    return "QAN " + str(df['QAN']) + " " + str(df['Result'])

In [7]:
#Function that calculates the English P8 score
def max_en(df):
    #Any student who has en and el the highest English grade will be doubled else it will not be doubled
    if pd.isnull(df['Open','Calc','Max English 2']):
        return df['P8 English','Calc','Max English 1']
    else:
        return df['P8 English','Calc','Max English 1'] * 2

In [8]:
'''Read all the Ebacc qualifcations and merge into a single dataframe'''

#All the EBacc qualifcation type (i.e tabs on the EBacc spreadsheet)
EBacc_slots = {
                'English':'English',
                'English Literature':'English',
                'Maths':'Maths',
                'Science':'EBacc',
                'Humanities':'EBacc',
                'Languages':'EBacc'
              }

#Stores all the Ebacc qualifcation data
EBacc_quals = []

#Loops through each relevant tab in the EBacc qual spreadsheet
for qual in EBacc_slots:
    
    #Read the sheet in the EBacc Quals spreadsheet
    qual_data = pd.read_excel('EBacc Quals.xlsx',sheet_name=qual)
    
    #Used to remove any header, footer and blank rows
    qual_data.dropna(subset=['Unnamed: 2'],inplace=True)
    
    #Rename the columns to the correct rows in the spreadsheet
    qual_data.columns = qual_data.iloc[0].tolist()
    
    #Drop the header column
    qual_data.drop([qual_data.index[0]],inplace=True)
    
    #Creates a column with which EBacc slot the dataframe of qualifcations belong to
    qual_data['P8 Slot'] = EBacc_slots[qual]
    
    EBacc_quals.append(qual_data)

#Puts all the EBacc data into 1 dataframe
EBacc_quals = pd.concat(EBacc_quals,sort=False)

#Remove any EBacc qualifcations that do not count in 2019
EBacc_quals.dropna(subset=[2019],inplace=True)

#Only need the QAN number and which EBacc slot the qualifcation belongs to
EBacc_quals = EBacc_quals[['QN','P8 Slot']]

In [9]:
#Reads the discount codes
discount_codes = pd.read_excel('Discount Codes.xlsx',sheet_name='2019',skiprows=[0,1])
#Enables the removal of footer columns
discount_codes.dropna(subset=['Qualification Title'],inplace=True)
#Only need the discount code and QAN columns
discount_codes = discount_codes[['Qualification Number','2019 KS4 Discount Code']]

In [10]:
#Gets the performace points.
performance_points = pd.read_excel('Performance Points.xlsx',sheet_name='Underlying Data',skiprows=[0,1,2])
#Enables the removal of footer columns
performance_points.dropna(subset=['Qualification Title'],inplace=True)

#Gets the AS data so that one can distinguish between level 3 and level 2 qualifications
AS_single = pd.read_excel('Performance Points.xlsx',sheet_name='AS Levels',skiprows=[0,1,2],usecols = [0,1])
AS_single.dropna(subset=['Qualification Title'],inplace=True)

AS_double = pd.read_excel('Performance Points.xlsx',sheet_name='AS Levels Double',skiprows=[0,1,2],usecols = [0,1])
AS_double.dropna(subset=['Qualification Title'],inplace=True)

AS = pd.concat([AS_single,AS_double])
AS['Qual Level'] = "KS5"

In [15]:
'''Create a U grade column for each qualification'''

#Create a pivot table with all the different qualifications
qual_lst = performance_points.pivot_table(index = ['Qualification Number','Qualification Title','Qualification Type',
                                         'GCSE Size Equivalence 1'],
                               values='Pass Points',aggfunc='first')
#Flatten the pivot table
qual_lst.reset_index(inplace=True)

,Qualification Number,Qualification Title,Qualification Type,GCSE Size Equivalence 1,Pass Points
0,10025480,OCR Level 3 Free Standing Mathematics Qualific...,Free standing Maths Qual Level 3,0.67,5.13
1,10034055,AQA Level 3 Advanced Subsidiary GCE in Mathema...,GCE AS level,1.00,10.75
2,10034110,Pearson Edexcel Level 3 Advanced Subsidiary GC...,GCE AS level,1.00,10.75
3,10034171,OCR Advanced Subsidiary GCE in Mathematics (MEI),GCE AS level,1.00,10.75
4,10034237,WJEC Advanced Subsidiary GCE in Mathematics,GCE AS level,1.00,10.75
5,10034298,CCEA Advanced Subsidiary GCE in Mathematics,GCE AS level,1.00,10.75
6,10034341,OCR Advanced Subsidiary GCE in Mathematics,GCE AS level,1.00,10.75
7,10034912,AQA Level 3 Advanced Subsidiary GCE in Statistics,GCE AS level,1.00,10.75
8,10042532,Pearson Edexcel Level 3 Advanced Subsidiary GC...,Applied GCE AS level,1.00,10.75
9,10042568,Pearson Edexcel Level 3 Advanced Subsidiary GC...,Applied GCE AS level,1.00,10.75


In [9]:
'''A merge column is created which is the QAN followed by the number in order to enable merging
    as merging in pandas does not work well with numbers
'''
discount_codes['QAN Merge'] = discount_codes.apply(lambda x : QAN_merge_col(x,'Qualification Number'),axis=1)
performance_points['QAN Merge'] = performance_points.apply(lambda x : QAN_merge_col(x,'Qualification Number'),axis=1)
EBacc_quals['QAN Merge'] = EBacc_quals.apply(lambda x : QAN_merge_col(x,'QN'),axis=1)
AS['QAN Merge'] = AS.apply(lambda x : QAN_merge_col(x,'Qualification Number'),axis=1)

In [10]:
discount_codes = discount_codes.merge(AS,how='left',on='QAN Merge')

#Any qualifcations that is not in the AS list we assume it is a KS4 qualification
discount_codes.fillna({'Qual Level':'KS4'},inplace=True)

In [11]:
'''Merge all the qualifcation data together into 1 dataframe'''
qualifcations = EBacc_quals.merge(performance_points,how='right',on='QAN Merge')
qualifcations = qualifcations.merge(discount_codes,how='left',on='QAN Merge')

#Any qualifcation that is not in the EBacc list should be classed as an open qualification
qualifcations.fillna({'P8 Slot':'Open'},inplace=True)

In [12]:
'''Create a column that enable the merging of the QAN 
    and the grade in order to get the points of each qualifcation'''
qualifcations['QAN Grade'] = qualifcations.apply(lambda x : QAN_grade_col(x),axis=1)

#Remove the coluumns that are not needed
qualifcations.drop(['QN','Grade Text','Qualification Number_x',
                    'Qualification Number_y','Qualification Title_y'],axis = 1,inplace=True)

#Rename the qualification title column
qualifcations.rename(columns={"Qualification Title_x": "Qualification Title"},inplace=True)

In [13]:
#gets the list of filenames with the subject results
results_filename=glob.glob('* Results.xlsx')

#stores the results data for each school
results = []

#loops through each file, gets the results and append to the list of results
for filename in results_filename:
    
    df = pd.read_excel(filename,sheet_name='data')
    results.append(df)
    
#Merege each of the school data together into 1 dataframe
results = pd.concat(results,sort=False)

#create a column that combines QAN with the student grade to enable merging
results['QAN Grade'] = results.apply(lambda x : QAN_student_grade_col(x),axis=1)

In [14]:
#Merge the data to get the equivalent DfE points and the discount codes
#Any qualifications that do not count will not be included
results = results.merge(qualifcations,on='QAN Grade',how='left')

In [15]:
#Sort the dataframe to enable removal of duplicate or discounting result
results.sort_values(by=['UPN','2019 KS4 Discount Code',
                          'Qual Level','QAN Merge',
                          'Result Date','Pass Points'],ascending=[True,True,False,True,True,False],inplace=True)

'''Delete results where student has sat the same Qual more than once.
    First attempt will remain unless qual was sat on the same day, then best attempt will remain
''' 
results.drop_duplicates(subset=['UPN','QAN Merge'],keep='first',inplace=True)

In [16]:
#Delete results with the same discount code. KS5 will superceed KS4
results.drop_duplicates(subset=['UPN','2019 KS4 Discount Code'],keep='first',inplace=True)

'''Gets a copy of all the science results. 
    This is so that the double science can be counted in two slots'''
scDouble = results[(results['2019 KS4 Discount Code']=='RA1E')].copy()

#Merge with data main dataframe and change qual name slightly
scDouble['Qualification Title'] = scDouble['Qualification Title'] + "B"

results = pd.concat([results,scDouble])

In [17]:
#Creates a pivot table to group columns by slot
Att8 = results.pivot_table(index = ['UPN'],columns = ['P8 Slot','2019 KS4 Discount Code','Qualification Title'],values='Pass Points',aggfunc='first')

In [18]:
#Gets the Maths points
maths = Att8.xs('Maths', level='P8 Slot', axis=1,drop_level=False).copy()

#Double the maths points for attainment 8
maths['P8 Maths','Calc','Max Maths 1']=maths.max(axis=1)*2

#Merge the Maths points for attainment 8 back to the main pivot table
Att8 = Att8.merge(maths.loc[:,['P8 Maths']],left_index=True, right_index=True,how="outer")

In [19]:
#Get the english points
english = Att8.xs('English', level='P8 Slot', axis=1,drop_level=False).copy()

#fill any NAN values with ~inf to enable doing the max using nlargest function
english['English'] = english['English'].fillna(float('-inf'))

#Calculate the highest English points
english['P8 English','Calc','Max English 1']= english.max(axis=1)

#Calculate the second highest English points which will count for the open slot
english['Open','Calc','Max English 2'] = english.loc[:,'English'].apply(lambda row: row.nlargest(2).values[-1],axis=1)

#replace ~inf with nan
english = english.replace(float('-inf'), np.nan)

#Any student who has en and el the highest English grade will be doubled else it will not be doubled
english['P8 English','Calc','Max English 1'] = english.apply(lambda x : max_en(x),axis=1)

english['P8 English','Calc'] = english['P8 English','Calc'].fillna(0)

#Merge the Max En points back to the main pivot table
Att8 = Att8.merge(english.loc[:,['P8 English','Open']],left_index=True, right_index=True,how="outer")


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2854: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2880: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


In [20]:
#Gets all the EBacc points
ebacc = Att8.xs('EBacc', level='P8 Slot', axis=1,drop_level=False).copy()

#fill any NAN values with ~inf to enable doing the max
ebacc['EBacc'] = ebacc['EBacc'].fillna(float('-inf'))

#Calculate the highest EBacc points
ebacc['P8 EBacc','Calc','Max EBacc 1']= ebacc.max(axis=1)

#Calculate the 2nd highest to the 6th highest EBacc points
for i in range(2, 7):
    if i<4:
        ebacc['P8 EBacc','Calc','Max EBacc '+ str(i)] = ebacc.loc[:,'EBacc'].apply(lambda row: row.nlargest(i).values[-1],axis=1)
    else:
        #4th to 6th highest EBacc points may count towards the open slots
        ebacc['Open','Calc','Max EBacc '+ str(i)] = ebacc.loc[:,'EBacc'].apply(lambda row: row.nlargest(i).values[-1],axis=1)

#replace ~inf with nan
ebacc = ebacc.replace(float('-inf'), np.nan)

#Merge the calculated Ebacc points back to the main pivot table
Att8 = Att8.merge(ebacc.loc[:,['P8 EBacc','Open']],left_index=True, right_index=True,how="outer")

In [21]:
#Gets all the Open points
open_slot = Att8.xs('Open', level='P8 Slot', axis=1,drop_level=False).copy()

#fill any NAN values with ~inf to enable doing the max
open_slot['Open'] = open_slot['Open'].fillna(float('-inf'))

#Calculate MAX Open 1
open_slot['P8 Open','Calc','Max Open 1'] = open_slot.max(axis=1)

#Calculate the 2nd highest and third highest open slot
for i in [2,3]:
    
    open_slot['P8 Open','Calc','Max Open '+ str(i)] = open_slot.loc[:,'Open'].apply(lambda row: row.nlargest(i).values[-1],axis=1)
    
#replace ~inf with nan
open_slot = open_slot.replace(float('-inf'), np.nan)

#Merge the calculated Ebacc points back to the main pivot table
Att8 = Att8.merge(open_slot.loc[:,['P8 Open']],left_index=True, right_index=True,how="outer")

In [22]:
#Flatten the pivot table using the Qualification title level or/and P8 slot name as the column name
Att8.columns = Att8.columns.get_level_values(2)

In [23]:
#Fill blanks with 0 to enable adding
Att8['Max Maths 1'].fillna(0)
Att8['Max English 1'].fillna(0)

for slot in ['EBacc','Open']:
    
    for i in [1,2,3]:
        
        Att8['Max {} {}'.format(slot,i)] = Att8['Max {} {}'.format(slot,i)].fillna(0)

In [24]:
#Calculate all the attainment 8 slots
Att8['Attainment 8 Maths'] = Att8['Max Maths 1']
Att8['Attainment 8 English'] = Att8['Max English 1']
Att8['Attainment 8 EBacc'] = Att8['Max EBacc 1']+ Att8['Max EBacc 2'] + Att8['Max EBacc 3']
Att8['Attainment 8 Open'] = Att8['Max Open 1'] + Att8['Max Open 2'] + Att8['Max Open 3']
Att8['Attainment 8 Score'] = Att8['Attainment 8 Maths'] + Att8['Attainment 8 English'] + Att8['Attainment 8 EBacc'] + Att8['Attainment 8 Open']

In [25]:
#Export the attainment 8 data to an excel file
Att8.to_excel("Attainment 8 Data.xlsx")